In [1]:
# Markov Chain Monte Carlo Maximum Likelihood Estimation
# Bayesian Model Comparison
# TODO: This is Metropolis algorithm. Rewrite for non-symmetric case (= Metropolis-Hastings) with bounds.
# Currently, bounds imposed by sigmoid regularization of the optimised variables

using HDF5, EzXML, Glob, Dates, Plots, Optim, Symbolics, JuMP, StatsBase, DataFrames, CSV, Random, Distributions, Printf
using CurveFit
include("/Users/jjc/CSF/CSF Dynamics Scripts/readCSF.jl")

datapath = "/Users/jjc/CSF/Recordings/"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);

global df = DataFrame(filename=files, Rcsf=NaN, E=NaN, Ib=NaN, P0=NaN, mode=NaN, error=NaN, R2=NaN, Pss=NaN, Rcsf_std=NaN, E_std=NaN, P0_std=NaN, Pss_std=NaN, Ib_std=NaN)

# for j = eachindex(files)
# for j  = 1:1000
# 990 good, 1141 good, 100 good, 331
for j = 100

    # j in [278, 442, 521, 621, 676, 708, 755, 989, 1010, 1041, 1042, 1043] ? (continue) : 0 # Some recordings do not have summaries data saved

    filename = files[j]
    # filename = "/Users/jjc/Documents/SSC/recs_june_2022/inf_20120717114717_INF2.hdf5"
    try
        global Data = readCSF(filename)
    catch
        continue
    end

    length(Data["ICP"]) > Data["infusion_end_frame"] ? (global Pm = Data["ICP"][Data["infusion_start_frame"]:Data["infusion_end_frame"]]) : (global Pm = Data["ICP"][Data["infusion_start_frame"]:end])
    global tslength = minimum([Data["infusion_end_frame"], length(Data["ICP"])])

    D = Pm # Data

    global lb = [0.01, 0.01, -10.0, 0.1]
    global ub = [50.0, 1.0, Data["P_b"], 0.9]
    Ib_lower = lb[4]
    Ib_upper = ub[4]

    θ₁ = zeros(4)
    try
    y = solvemodel(θ₁...)
    catch
        continue
    end
    y = y[Data["infusion_start_frame"]:end]

    σ = 1.0 # standard deviation for the model 
    χ₁ = sum((D .- y) .^ 2) / (2 * σ^2) #

    total_n = 50000 # Number of iterations
    burntime = 10000 # Burn time

    # total_n = 1000000 # Number of iterations
    # burntime = 100000 # Burn time
    # a = [10.35, 0.14, 10.54] # Guess standard deviation
    # a = zeros(3) .+ 0.1 # Guess standard deviation for the parameters
    a = zeros(4) .+ 0.1 # Guess standard deviation for the parameters = how wide around the current guess we want to deviate [0.01 good], higher = exploration, lower = exploitation
    # TODO: consider adding a different one to the I_b convergence to bounds

    global thetasave = zeros(4, total_n - burntime)
    global thetasave_t = zeros(4, total_n - burntime)
    global chisave = zeros(total_n - burntime)

    for i = 1:total_n

        θ₂ = θ₁ .+ a .* randn(4) # Normal distribution

        Rcsf = sigmoid(θ₁[1], lb[1], ub[1])
        E = sigmoid(θ₁[2], lb[2], ub[2])
        P_0 = sigmoid(Data["P_0"])
        Pss = sigmoid(θ₁[4], lb[3], ub[3])
        I_b = (Data["P_b"] - Pss) / Rcsf

        while (I_b < lb[4]) || (I_b > ub[4])
            θ₂ = θ₁ .+ a .* randn(4) # Normal distribution
            Rcsf = sigmoid(θ₂[1], lb[1], ub[1])
            E = sigmoid(θ₂[2], lb[2], ub[2])
            P_0 = sigmoid(θ₂[3], lb[3], ub[3])
            Pss = sigmoid(θ₂[4], lb[3], ub[3])
            I_b = (Data["P_b"] - Pss) / Rcsf
        end

        y = solvemodel(θ₂...)
        y = y[Data["infusion_start_frame"]:end]
        χ₂ = sum((D .- y) .^ 2) / (2 * σ^2)

        ratio = exp((-χ₂ + χ₁) / 2 * σ^2) # Likelihood ratio
        r = rand(1)[1] # Uniform distribution

        if r < ratio
            θ₁ = θ₂
            χ₁ = χ₂
        end

        ths = zeros(4)
        if i > burntime
            ths[1] = sigmoid(θ₁[1], lb[1], ub[1])
            ths[2] = sigmoid(θ₁[2], lb[2], ub[2])
            ths[3] = sigmoid(θ₁[3], lb[3], ub[3])
            ths[4] = sigmoid(θ₁[4], lb[3], ub[3])
            thetasave[:, i-burntime] = θ₁
            thetasave_t[:, i-burntime] .= ths
            chisave[i-burntime] = χ₁
        end
    end

    # thetasave_t = thetasave

    Ib_all = zeros(total_n - burntime)
    global Ib_all .= (Data["P_b"] .- thetasave_t[4, :]) ./ thetasave_t[1, :]
    # global Ib_all .= (Data["P_b"] .- Data["P_0"]) ./ thetasave_t[1, :]

    # println("Rcsf = $(round(mean(thetasave_t[1,:]),digits=2)) ± $(round(std(thetasave_t[1,:]),digits=2))")
    # println("I_b = $(round(mean(Ib_all),digits=2)) ± $(round(std(Ib_all),digits=2))")
    # println("E = $(round(mean(thetasave_t[2,:]),digits=2)) ± $(round(std(thetasave_t[2,:]),digits=2))")
    # println("P_0 = $(round(mean(thetasave_t[3,:]),digits=2)) ± $(round(std(thetasave_t[3,:]),digits=2))")

    # println("I_b = $(round(median(Ib_all),digits=2)) ± $(round(std(Ib_all),digits=2))")
    # println("Rcsf = $(round(median(thetasave_t[1,:]),digits=2)) ± $(round(std(thetasave_t[1,:]),digits=2))")
    # println("E = $(round(median(thetasave_t[2,:]),digits=2)) ± $(round(std(thetasave_t[2,:]),digits=2))")
    # println("P_0 = $(round(median(thetasave_t[3,:]),digits=2)) ± $(round(std(thetasave_t[3,:]),digits=2))")
    # println("Pss = $(round(median(thetasave_t[4,:]),digits=2)) ± $(round(std(thetasave_t[4,:]),digits=2))")

    opt_idx = findmin(chisave)[2]
    Rcsf = sigmoid(thetasave[1, opt_idx], lb[1], ub[1])
    E = sigmoid(thetasave[2, opt_idx], lb[2], ub[2])
    P_0 = sigmoid(thetasave[3, opt_idx], lb[3], ub[3])
    Pss = sigmoid(thetasave[4, opt_idx], lb[3], ub[3])
    I_b = (Data["P_b"] - Pss) / Rcsf

    # df.Rcsf[j] = (round(mean(thetasave_t[1, :]), digits=2))
    # df.P0[j] = (round(mean(thetasave_t[3, :]), digits=2))
    # df.Pss[j] = (round(mean(thetasave_t[4, :]), digits=2))
    # df.Ib[j] = (round(mean(Ib_all), digits=2))
    # df.E[j] = (round(mean(thetasave_t[2, :]), digits=2))

    df.Rcsf[j] = round(Rcsf, digits=2)
    df.P0[j] = round(P_0, digits=2)
    df.Pss[j] = round(Pss, digits=2)
    df.Ib[j] = round(I_b, digits=2)
    df.E[j] = round(E, digits=2)

    df.filename[j] = filename[length(datapath)+2:end]
    df.error[j] = round(calc_model_plot(I_b, E, P_0, Pss)[2], digits=3)
    # global Pm = solvemodel(Rcsf, E, P_0)
    length(Data["ICP"]) > Data["infusion_end_frame"] ? (global Pm = Data["ICP"][Data["infusion_start_frame"]:Data["infusion_end_frame"]]) : (global Pm = Data["ICP"][Data["infusion_start_frame"]:end])
    global tslength = minimum([Data["infusion_end_frame"], length(Data["ICP"])])
    volRes, pressRes, fitted_curve, R2, MSE = press_vol_curve(Rcsf, P_0)
    df.R2[j] = round(R2, digits=3)
    df.Rcsf_std[j] = (round(std(thetasave_t[1, :]), digits=2))
    df.E_std[j] = (round(std(thetasave_t[2, :]), digits=2))
    df.P0_std[j] = (round(std(thetasave_t[3, :]), digits=2))
    df.Pss_std[j] = (round(std(thetasave_t[4, :]), digits=2))
    df.Ib_std[j] = (round(std(Ib_all), digits=2))
    println("File $j / $(length(files))")
    # CSV.write("/Users/jjc/CSF/Results/Results_BayesSTD.csv", df)
end
# CSV.write("/Users/jjc/CSF/Results/Results_Bayes4params.csv", df)
μ = mean(thetasave_t, dims=2)
σ = std(thetasave_t, dims=2)

opt_idx = findmin(chisave)[2]
Rcsf = sigmoid(thetasave[1, opt_idx], lb[1], ub[1])
E = sigmoid(thetasave[2, opt_idx], lb[2], ub[2])
P_0 = sigmoid(thetasave[3, opt_idx], lb[3], ub[3])
Pss = sigmoid(thetasave[4, opt_idx], lb[3], ub[3])

Rcsf_std = std(thetasave_t[1, :])
E_std = std(thetasave_t[2, :])
P0_std = std(thetasave_t[3, :])
Pss_std = std(thetasave_t[4, :])
Ib_std = std(Ib_all)

# Rcsf = sigmoid(mean(thetasave[1,:]), lb[1], ub[1])
# E = sigmoid(mean(thetasave[2,:]), lb[2], ub[2])
# P_0 = sigmoid(mean(thetasave[3,:]), lb[3], ub[3])
# Pss = sigmoid(mean(thetasave[4,:]), lb[3], ub[3])

# I_b = (Data["P_b"] - Pss) / Rcsf
# rmserr = calc_model_plot(I_b, E, P_0, Pss)[2]
# h0 = plotmodel(I_b, E, P_0, Pss, μ, σ, "dar", "4param")

# title!(
#     @sprintf("I_b = %.2f ± %.2f [mL/min]\n", I_b, Ib_std) *
#     @sprintf("Rcsf = %.2f ± %.2f [mmHg/(ml/min)]\n", Rcsf, Rcsf_std) *
#     @sprintf("E = %.2f ± %.2f [1/mL]\n", E, E_std) *
#     @sprintf("P_0 = %.2f ± %.2f [mmHg]\n", P_0, P0_std) *
#     @sprintf("error = %.4f", rmserr),
#     titlealign=:left, titlefontsize=8, xlabel="Time [min]", ylabel="ICP [mmHg]"
# )
# icmp_curve = calc_model_plot((Data["P_b"]-Data["P_0"])/Data["Rcsf"], Data["E"], Data["P_0"])[1]
# plot!(icmp_curve, lw=2, alpha=0.7, label="∇ descent")

File 100 / 1141


0.03261997104579287